In [1]:
import pandas as pd
import io
import requests
import numpy as np
import os
import matplotlib.pyplot as plt
import pylab as pl
import tensorflow.contrib.learn as skflow
import string

from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping
from sklearn import preprocessing
%matplotlib inline

path = "UNSW-NB15.csv"
# This file is a CSV, just no CSV extension or headers
df = pd.read_csv(path, header=None)
print(df[0:3])

print("Read {} rows.".format(len(df)))
# df = df.sample(frac=0.1, replace=False) # Uncomment this line to sample only 10% of the dataset
df.dropna(inplace=True,axis=1) # For now, just drop NA's (rows with missing values)

# The CSV file has no column heads, so add them
df.columns = [
    'srcip',
    'sport',
    'dstip',
    'dsport',
    'proto',
    'state',
    'sbyte',
    'dbytes',
    'sttl',
    'dttl',
    'sloss',
    'dloss',
    'service',
    'Sload',
    'Dload',
    'Spkts',
    'Dpkts',
    'swin',
    'dwin',
    'stcpb',
    'dtcpb',
    'smeansz',
    'dmeansz',
    'trans',
    'res',
    'Sjit',
    'Djit',
    'Stime', #QUITAR
    'Ltime', #QUITAR
    'Sintpkt',
    'Dintpkt',
    'tcprtt',
    'synack',
    'ackdat',
    'is_sm_ips_ports',
    'ct_state_ttl',
    'ct_flw_http_mthd',
    'is_ftp_login',
    'ct_ftp_cmd',
    'ct_srv_src',
    'ct_srv_dst',
    'ct_dst_ltm',
    'ct_src_ltm', 
    'ct_src_dport_ltm',
    'ct_dst_sport_lt', 
    'ct_dst_src_ltm',
    'attack_cat',
    'Label'

]

#df.drop('algo_borrar', 1, inplace=True)

ENCODING = 'utf-8'

def expand_categories(values):
    result = []
    s = values.value_counts()
    t = float(len(values))
    for v in s.index:
        result.append("{}:{}%".format(v,round(100*(s[v]/t),2)))
    return "[{}]".format(",".join(result))
        
def analyze(filename):
    print()
    print("Analyzing: {}".format(filename))
    df = pd.read_csv(filename,encoding=ENCODING)
    cols = df.columns.values
    total = float(len(df))

    print("{} rows".format(int(total)))
    for col in cols:
        uniques = df[col].unique()
        unique_count = len(uniques)
        if unique_count>100:
            print("** {}:{} ({}%)".format(col,unique_count,int(((unique_count)/total)*100)))
        else:
            print("** {}:{}".format(col,expand_categories(df[col])))
            expand_categories(df[col])

analyze(path)


# display 5 rows
df[0:5]

# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = f"{name}-{x}"
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)

#Clean 'date' column and convert to Int type
def clean_date(s):
    s = ''.join([i for i in s if i not in frozenset(string.punctuation)])
    s_removed = s.replace(" ", "")
    s_int = int(s_removed)
    return s_int

########## CLEAN IP #######################
def clean_ip(s):
    s = ''.join([i for i in s if i not in frozenset(string.punctuation)])
    s_int = int(s)
    return s_int

# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd

#MINMAX -1 1
# Encode a column to a range between normalized_low and normalized_high.
def min_max_1(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
        * (normalized_high - normalized_low) + normalized_low

#MINMAX 0 1
def min_max_0(df, name, normalized_low=0, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
        * (normalized_high - normalized_low) + normalized_low

    
# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_

# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(
        target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    # Regression
    return df[result].values.astype(np.float32), df[[target]].values.astype(np.float32)

# Plot a confusion matrix.
# cm is the confusion matrix, names are the names of the classes.
def plot_confusion_matrix(cm, names, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(names))
    plt.xticks(tick_marks, names, rotation=45)
    plt.yticks(tick_marks, names)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')




df['srcip'] = df['srcip'].apply(clean_ip)
df['dstip'] = df['dstip'].apply(clean_ip)

# Now encode the feature vector

encode_numeric_zscore((df, 'srcip')#
#encode_numeric_zscore((df, 'sport')
df.drop('sport', 1, inplace=True)#
encode_numeric_zscore(df, 'dstip') #
#encode_numeric_zscore(df, 'dsport') #must be str not int ???
df.drop('dsport', 1, inplace=True)#
encode_text_dummy(df, 'proto') #
#encode_text_dummy(df, 'state')
df.drop('state', 1, inplace=True)#
#encode_numeric_zscore(df, 'dur')
df.drop('dur', 1, inplace=True)#
encode_numeric_zscore(df, 'sbyte') #
encode_numeric_zscore(df, 'dbytes')#
encode_numeric_zscore(df, 'sttl')#
encode_numeric_zscore(df, 'dttl')#
encode_numeric_zscore(df, 'sloss')#
encode_numeric_zscore(df, 'dloss')#
encode_text_dummy(df, 'service') #
#encode_numeric_zscore(df, 'Sload')
df.drop('Sload', 1, inplace=True)#
#encode_numeric_zscore(df, 'Dload')
df.drop('Dload', 1, inplace=True)#
encode_numeric_zscore(df, 'Spkts')#
encode_numeric_zscore(df, 'Dpkts')#
encode_numeric_zscore(df, 'swin')#
encode_numeric_zscore(df, 'dwin')#
encode_numeric_zscore(df, 'stcpb')#
encode_numeric_zscore(df, 'dtcpb')#
encode_numeric_zscore(df, 'smeansz')#
encode_numeric_zscore(df, 'dmeansz')#
encode_numeric_zscore(df, 'trans')#
encode_numeric_zscore(df, 'res')#
encode_numeric_zscore(df, 'Sjit')#
encode_numeric_zscore(df, 'Djit')#
df.drop('Stime', 1, inplace=True)#
df.drop('Ltime', 1, inplace=True)#
encode_numeric_zscore(df, 'Sintpkt')#
encode_numeric_zscore(df, 'Dintpkt')#
encode_numeric_zscore(df, 'tcprtt')#
encode_numeric_zscore(df, 'synack')#
encode_numeric_zscore(df, 'ackdat')#
#encode_numeric_zscore(df, 'is_sm_ips_ports')#
encode_numeric_zscore(df, 'ct_state_ttl')#
encode_numeric_zscore(df, 'ct_flw_http_mthd')#
#encode_numeric_zscore(df, 'is_ftp_login')#
encode_numeric_zscore(df, 'ct_ftp_cmd')#
encode_numeric_zscore(df, 'ct_srv_src')#
encode_numeric_zscore(df, 'ct_srv_dst')#
encode_numeric_zscore(df, 'ct_dst_ltm')#
#encode_numeric_zscore(df, 'ct_src_ltm')
df.drop('ct_src_ltm', 1, inplace=True)#
encode_numeric_zscore(df, 'ct_src_dport_ltm')#
encode_numeric_zscore(df, 'ct_dst_sport_lt')#
encode_numeric_zscore(df, 'ct_dst_src_ltm')#
encode_numeric_zscore(df, 'attack_cat')#


outcomes = encode_text_index(df, 'Label')#
num_classes = len(outcomes)

# display 5 rows

df.dropna(inplace=True,axis=1)
df[0:5]

SyntaxError: invalid syntax (<ipython-input-1-87b2e6743d82>, line 219)

In [2]:
print(df['sport'])

NameError: name 'df' is not defined

In [5]:
import pandas as pd
import io
import requests
import numpy as np
import os
import matplotlib.pyplot as plt
import pylab as pl
import tensorflow.contrib.learn as skflow
import string

from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping
from sklearn import preprocessing
%matplotlib inline

path = "UNSW-NB15_1.csv"
# This file is a CSV, just no CSV extension or headers
df = pd.read_csv(path, header=None)
print(df[0:3])

print("Read {} rows.".format(len(df)))
# df = df.sample(frac=0.1, replace=False) # Uncomment this line to sample only 10% of the dataset
df.dropna(inplace=True,axis=1) # For now, just drop NA's (rows with missing values)

# The CSV file has no column heads, so add them
df.columns = [
    'srcip',
    'sport',
    'dstip',
    'dsport',
    'proto',
    'state',
    'sbyte',
    'dbytes',
    'sttl',
    'dttl',
    'sloss',
    'dloss',
    'service',
    'Sload',
    'Dload',
    'Spkts',
    'Dpkts',
    'swin',
    'dwin',
    'stcpb',
    'dtcpb',
    'smeansz',
    'dmeansz',
    'trans',
    'res',
    'Sjit',
    'Djit',
    'Stime', #QUITAR
    'Ltime', #QUITAR
    'Sintpkt',
    'Dintpkt',
    'tcprtt',
    'synack',
    'ackdat',
    'is_sm_ips_ports',
    'ct_state_ttl',
    'ct_flw_http_mthd',
    'is_ftp_login',
    'ct_ftp_cmd',
    'ct_srv_src',
    'ct_srv_dst',
    'ct_dst_ltm',
    'ct_src_ltm', 
    'ct_src_dport_ltm',
    'ct_dst_sport_lt', 
    'ct_dst_src_ltm',
    'attack_cat',
    'Label'

]

/miniconda3/envs/tf_jupyter/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


           0      1              2     3    4    5         6    7    8   9   \
0  59.166.0.0   1390  149.171.126.6    53  udp  CON  0.001055  132  164  31   
1  59.166.0.0  33661  149.171.126.9  1024  udp  CON  0.036133  528  304  31   
2  59.166.0.6   1464  149.171.126.7    53  udp  CON  0.001119  146  178  31   

   ...  39  40  41 42  43  44  45  46   47  48  
0  ...   0   3   7  1   3   1   1   1  NaN   0  
1  ...   0   2   4  2   3   1   1   2  NaN   0  
2  ...   0  12   8  1   2   2   1   1  NaN   0  

[3 rows x 49 columns]
Read 700001 rows.


In [4]:
pwd

'/Users/beatrizesteban/Desktop/csv_tfg/UNSW-NB15'

In [6]:
ENCODING = 'utf-8'

def expand_categories(values):
    result = []
    s = values.value_counts()
    t = float(len(values))
    for v in s.index:
        result.append("{}:{}%".format(v,round(100*(s[v]/t),2)))
    return "[{}]".format(",".join(result))
        
def analyze(filename):
    print()
    print("Analyzing: {}".format(filename))
    df = pd.read_csv(filename,encoding=ENCODING)
    cols = df.columns.values
    total = float(len(df))

    print("{} rows".format(int(total)))
    for col in cols:
        uniques = df[col].unique()
        unique_count = len(uniques)
        if unique_count>100:
            print("** {}:{} ({}%)".format(col,unique_count,int(((unique_count)/total)*100)))
        else:
            print("** {}:{}".format(col,expand_categories(df[col])))
            expand_categories(df[col])

analyze(path)


# display 5 rows
df[0:5]

# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = f"{name}-{x}"
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)

#Clean 'date' column and convert to Int type
def clean_date(s):
    s = ''.join([i for i in s if i not in frozenset(string.punctuation)])
    s_removed = s.replace(" ", "")
    s_int = int(s_removed)
    return s_int

########## CLEAN IP #######################
def clean_ip(s):
    s = ''.join([i for i in s if i not in frozenset(string.punctuation)])
    s_int = int(s)
    return s_int

# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd

#MINMAX -1 1
# Encode a column to a range between normalized_low and normalized_high.
def min_max_1(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
        * (normalized_high - normalized_low) + normalized_low

#MINMAX 0 1
def min_max_0(df, name, normalized_low=0, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
        * (normalized_high - normalized_low) + normalized_low

    
# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_

# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(
        target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    # Regression
    return df[result].values.astype(np.float32), df[[target]].values.astype(np.float32)

# Plot a confusion matrix.
# cm is the confusion matrix, names are the names of the classes.
def plot_confusion_matrix(cm, names, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(names))
    plt.xticks(tick_marks, names, rotation=45)
    plt.yticks(tick_marks, names)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')




df['srcip'] = df['srcip'].apply(clean_ip)
df['dstip'] = df['dstip'].apply(clean_ip)

# Now encode the feature vector

encode_numeric_zscore((df, 'srcip')#
#encode_numeric_zscore((df, 'sport')
df.drop('sport', 1, inplace=True)#
encode_numeric_zscore(df, 'dstip') #
#encode_numeric_zscore(df, 'dsport') #must be str not int ???
df.drop('dsport', 1, inplace=True)#
encode_text_dummy(df, 'proto') #
#encode_text_dummy(df, 'state')
df.drop('state', 1, inplace=True)#
#encode_numeric_zscore(df, 'dur')
df.drop('dur', 1, inplace=True)#
encode_numeric_zscore(df, 'sbyte') #
encode_numeric_zscore(df, 'dbytes')#
encode_numeric_zscore(df, 'sttl')#
encode_numeric_zscore(df, 'dttl')#
encode_numeric_zscore(df, 'sloss')#
encode_numeric_zscore(df, 'dloss')#
encode_text_dummy(df, 'service') #
#encode_numeric_zscore(df, 'Sload')
df.drop('Sload', 1, inplace=True)#
#encode_numeric_zscore(df, 'Dload')
df.drop('Dload', 1, inplace=True)#
encode_numeric_zscore(df, 'Spkts')#
encode_numeric_zscore(df, 'Dpkts')#
encode_numeric_zscore(df, 'swin')#
encode_numeric_zscore(df, 'dwin')#
encode_numeric_zscore(df, 'stcpb')#
encode_numeric_zscore(df, 'dtcpb')#
encode_numeric_zscore(df, 'smeansz')#
encode_numeric_zscore(df, 'dmeansz')#
encode_numeric_zscore(df, 'trans')#
encode_numeric_zscore(df, 'res')#
encode_numeric_zscore(df, 'Sjit')#
encode_numeric_zscore(df, 'Djit')#
df.drop('Stime', 1, inplace=True)#
df.drop('Ltime', 1, inplace=True)#
encode_numeric_zscore(df, 'Sintpkt')#
encode_numeric_zscore(df, 'Dintpkt')#
encode_numeric_zscore(df, 'tcprtt')#
encode_numeric_zscore(df, 'synack')#
encode_numeric_zscore(df, 'ackdat')#
#encode_numeric_zscore(df, 'is_sm_ips_ports')#
encode_numeric_zscore(df, 'ct_state_ttl')#
encode_numeric_zscore(df, 'ct_flw_http_mthd')#
#encode_numeric_zscore(df, 'is_ftp_login')#
encode_numeric_zscore(df, 'ct_ftp_cmd')#
encode_numeric_zscore(df, 'ct_srv_src')#
encode_numeric_zscore(df, 'ct_srv_dst')#
encode_numeric_zscore(df, 'ct_dst_ltm')#
#encode_numeric_zscore(df, 'ct_src_ltm')
df.drop('ct_src_ltm', 1, inplace=True)#
encode_numeric_zscore(df, 'ct_src_dport_ltm')#
encode_numeric_zscore(df, 'ct_dst_sport_lt')#
encode_numeric_zscore(df, 'ct_dst_src_ltm')#
encode_numeric_zscore(df, 'attack_cat')#


outcomes = encode_text_index(df, 'Label')#
num_classes = len(outcomes)

# display 5 rows

df.dropna(inplace=True,axis=1)
df[0:5]

SyntaxError: invalid syntax (<ipython-input-6-1ea103224931>, line 134)

In [7]:
print(df['srcip'])

0           59.166.0.0
1           59.166.0.0
2           59.166.0.6
3           59.166.0.5
4           59.166.0.3
5           59.166.0.0
6           59.166.0.6
7          10.40.182.3
8           59.166.0.5
9           59.166.0.7
10         10.40.170.2
11         10.40.170.2
12         10.40.182.3
13          59.166.0.1
14          59.166.0.1
15          59.166.0.1
16          59.166.0.2
17          59.166.0.1
18          59.166.0.1
19          59.166.0.4
20        175.45.176.3
21        175.45.176.2
22        175.45.176.0
23          59.166.0.3
24          59.166.0.2
25          59.166.0.8
26          59.166.0.9
27          59.166.0.0
28          59.166.0.5
29          59.166.0.7
              ...     
699971      59.166.0.1
699972      59.166.0.7
699973      59.166.0.9
699974      59.166.0.0
699975      59.166.0.3
699976      59.166.0.2
699977      59.166.0.3
699978      59.166.0.3
699979      59.166.0.8
699980      59.166.0.4
699981      59.166.0.1
699982      59.166.0.2
699983     

In [8]:
print(df['sport'])

0          1390
1         33661
2          1464
3          3593
4         49664
5         32119
6          2142
7             0
8         40726
9         12660
10            0
11            0
12            0
13        48847
14        24266
15        10393
16        62539
17        21270
18         8989
19        49346
20        21223
21        23357
22        13284
23         4192
24        26872
25        24946
26         5685
27        22848
28        28565
29        46719
          ...  
699971    46807
699972    60323
699973    65260
699974    36648
699975    14404
699976    60849
699977    14472
699978    18827
699979    24528
699980    19461
699981    35327
699982     9969
699983    33100
699984    12691
699985    18895
699986    21661
699987    10213
699988    31786
699989    52262
699990    28517
699991    34947
699992    48114
699993    48812
699994    48114
699995     8543
699996    12520
699997    18895
699998    30103
699999    30388
700000     6055
Name: sport, Length: 700

In [9]:
df.drop('sport', 1, inplace=True)

In [10]:
clear

In [12]:
import pandas as pd
import io
import requests
import numpy as np
import os
import matplotlib.pyplot as plt
import pylab as pl
import tensorflow.contrib.learn as skflow
import string

from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping
from sklearn import preprocessing
%matplotlib inline

path = "UNSW-NB15_1.csv"
# This file is a CSV, just no CSV extension or headers
df = pd.read_csv(path, header=None)
print(df[0:3])

print("Read {} rows.".format(len(df)))
# df = df.sample(frac=0.1, replace=False) # Uncomment this line to sample only 10% of the dataset
df.dropna(inplace=True,axis=1) # For now, just drop NA's (rows with missing values)

# The CSV file has no column heads, so add them
df.columns = [
    'srcip',
    'sport',
    'dstip',
    'dsport',
    'proto',
    'state',
    'sbyte',
    'dbytes',
    'sttl',
    'dttl',
    'sloss',
    'dloss',
    'service',
    'Sload',
    'Dload',
    'Spkts',
    'Dpkts',
    'swin',
    'dwin',
    'stcpb',
    'dtcpb',
    'smeansz',
    'dmeansz',
    'trans',
    'res',
    'Sjit',
    'Djit',
    'Stime', #QUITAR
    'Ltime', #QUITAR
    'Sintpkt',
    'Dintpkt',
    'tcprtt',
    'synack',
    'ackdat',
    'is_sm_ips_ports',
    'ct_state_ttl',
    'ct_flw_http_mthd',
    'is_ftp_login',
    'ct_ftp_cmd',
    'ct_srv_src',
    'ct_srv_dst',
    'ct_dst_ltm',
    'ct_src_ltm', 
    'ct_src_dport_ltm',
    'ct_dst_sport_lt', 
    'ct_dst_src_ltm',
    'attack_cat',
    'Label'

]

df.drop('sport', 1, inplace=True)#
df.drop('dsport', 1, inplace=True)#
df.drop('state', 1, inplace=True)#
df.drop('dur', 1, inplace=True)#
df.drop('Sload', 1, inplace=True)#
df.drop('Dload', 1, inplace=True)#
df.drop('Stime', 1, inplace=True)#
df.drop('Ltime', 1, inplace=True)#
df.drop('ct_src_ltm', 1, inplace=True)#

/miniconda3/envs/tf_jupyter/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


           0      1              2     3    4    5         6    7    8   9   \
0  59.166.0.0   1390  149.171.126.6    53  udp  CON  0.001055  132  164  31   
1  59.166.0.0  33661  149.171.126.9  1024  udp  CON  0.036133  528  304  31   
2  59.166.0.6   1464  149.171.126.7    53  udp  CON  0.001119  146  178  31   

   ...  39  40  41 42  43  44  45  46   47  48  
0  ...   0   3   7  1   3   1   1   1  NaN   0  
1  ...   0   2   4  2   3   1   1   2  NaN   0  
2  ...   0  12   8  1   2   2   1   1  NaN   0  

[3 rows x 49 columns]
Read 700001 rows.


KeyError: "['dur'] not found in axis"

In [13]:
import pandas as pd
import io
import requests
import numpy as np
import os
import matplotlib.pyplot as plt
import pylab as pl
import tensorflow.contrib.learn as skflow
import string

from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping
from sklearn import preprocessing
%matplotlib inline

path = "UNSW-NB15_1.csv"
# This file is a CSV, just no CSV extension or headers
df = pd.read_csv(path, header=None)
print(df[0:3])

print("Read {} rows.".format(len(df)))
# df = df.sample(frac=0.1, replace=False) # Uncomment this line to sample only 10% of the dataset
df.dropna(inplace=True,axis=1) # For now, just drop NA's (rows with missing values)

# The CSV file has no column heads, so add them
df.columns = [
    'srcip',
    'sport',
    'dstip',
    'dsport',
    'proto',
    'state',
    'dur',
    'sbyte',
    'dbytes',
    'sttl',
    'dttl',
    'sloss',
    'dloss',
    'service',
    'Sload',
    'Dload',
    'Spkts',
    'Dpkts',
    'swin',
    'dwin',
    'stcpb',
    'dtcpb',
    'smeansz',
    'dmeansz',
    'trans',
    'res',
    'Sjit',
    'Djit',
    'Stime', #QUITAR
    'Ltime', #QUITAR
    'Sintpkt',
    'Dintpkt',
    'tcprtt',
    'synack',
    'ackdat',
    'is_sm_ips_ports',
    'ct_state_ttl',
    'ct_flw_http_mthd',
    'is_ftp_login',
    'ct_ftp_cmd',
    'ct_srv_src',
    'ct_srv_dst',
    'ct_dst_ltm',
    'ct_src_ltm', 
    'ct_src_dport_ltm',
    'ct_dst_sport_lt', 
    'ct_dst_src_ltm',
    'attack_cat',
    'Label'

]

df.drop('sport', 1, inplace=True)#
df.drop('dsport', 1, inplace=True)#
df.drop('state', 1, inplace=True)#
df.drop('dur', 1, inplace=True)#
df.drop('Sload', 1, inplace=True)#
df.drop('Dload', 1, inplace=True)#
df.drop('Stime', 1, inplace=True)#
df.drop('Ltime', 1, inplace=True)#
df.drop('ct_src_ltm', 1, inplace=True)#

           0      1              2     3    4    5         6    7    8   9   \
0  59.166.0.0   1390  149.171.126.6    53  udp  CON  0.001055  132  164  31   
1  59.166.0.0  33661  149.171.126.9  1024  udp  CON  0.036133  528  304  31   
2  59.166.0.6   1464  149.171.126.7    53  udp  CON  0.001119  146  178  31   

   ...  39  40  41 42  43  44  45  46   47  48  
0  ...   0   3   7  1   3   1   1   1  NaN   0  
1  ...   0   2   4  2   3   1   1   2  NaN   0  
2  ...   0  12   8  1   2   2   1   1  NaN   0  

[3 rows x 49 columns]
Read 700001 rows.


ValueError: Length mismatch: Expected axis has 48 elements, new values have 49 elements